In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.ticker as ticker
from IPython.display import Audio
from sklearn import datasets, linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

In [2]:
#Billboard Top 100 By Week from 1958 to 2021
bb100 = pd.read_csv('Hot Stuff.csv')

In [3]:
#First & Last 2 entries by Week/Year
bb100['WeekID'] = pd.to_datetime(bb100['WeekID'])

In [4]:
#Features
features = pd.read_excel('Hot 100 Audio Features.xlsx')

In [5]:
bb_all = pd.merge(bb100, features, on='SongID', how='left')
bb_all

,url,WeekID,Week Position,Song_x,Performer_x,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,Performer_y,Song_y,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,http://www.billboard.com/charts/hot-100/1965-07-17,1965-07-17,34,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,45.0,34,4,Patty Duke,Don't Just Stand There,['deep adult standards'],1YhNCQ3XOdTCZgubfX8PgB,NaN,163160.0,0.0,Lost Hits Of The 60's (All Original Artists & Versions),0.574,0.256,7.0,-15.044,1.0,0.0298,0.6100,0.000077,0.1000,0.568,82.331,3.0,21.0
1,http://www.billboard.com/charts/hot-100/1965-07-24,1965-07-24,22,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,34.0,22,5,Patty Duke,Don't Just Stand There,['deep adult standards'],1YhNCQ3XOdTCZgubfX8PgB,NaN,163160.0,0.0,Lost Hits Of The 60's (All Original Artists & Versions),0.574,0.256,7.0,-15.044,1.0,0.0298,0.6100,0.000077,0.1000,0.568,82.331,3.0,21.0
2,http://www.billboard.com/charts/hot-100/1965-07-31,1965-07-31,14,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,22.0,14,6,Patty Duke,Don't Just Stand There,['deep adult standards'],1YhNCQ3XOdTCZgubfX8PgB,NaN,163160.0,0.0,Lost Hits Of The 60's (All Original Artists & Versions),0.574,0.256,7.0,-15.044,1.0,0.0298,0.6100,0.000077,0.1000,0.568,82.331,3.0,21.0
3,http://www.billboard.com/charts/hot-100/1965-08-07,1965-08-07,10,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,14.0,10,7,Patty Duke,Don't Just Stand There,['deep adult standards'],1YhNCQ3XOdTCZgubfX8PgB,NaN,163160.0,0.0,Lost Hits Of The 60's (All Original Artists & Versions),0.574,0.256,7.0,-15.044,1.0,0.0298,0.6100,0.000077,0.1000,0.568,82.331,3.0,21.0
4,http://www.billboard.com/charts/hot-100/1965-08-14,1965-08-14,8,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,10.0,8,8,Patty Duke,Don't Just Stand There,['deep adult standards'],1YhNCQ3XOdTCZgubfX8PgB,NaN,163160.0,0.0,Lost Hits Of The 60's (All Original Artists & Versions),0.574,0.256,7.0,-15.044,1.0,0.0298,0.6100,0.000077,0.1000,0.568,82.331,3.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330456,https://www.billboard.com/charts/hot-100/2018-10-20,2018-10-20,22,God Is A Woman,Ariana Grande,God Is A WomanAriana Grande,1,21.0,8,13,Ariana Grande,God Is A Woman,"['dance pop', 'pop', 'post-teen pop']",67kQ3UW7saqCFqB54BVKKI,NaN,213068.0,0.0,k bye for now (swt live),0.602,0.658,1.0,-5.934,1.0,0.0558,0.0233,0.000060,0.2370,0.268,145.031,4.0,62.0
330457,http://www.billboard.com/charts/hot-100/1977-05-21,1977-05-21,22,I Wanna Get Next To You,Rose Royce,I Wanna Get Next To YouRose Royce,1,10.0,10,13,Rose Royce,I Wanna Get Next To You,"['classic soul', 'disco', 'funk', 'motown', 'post-disco', 'quiet storm', 'soul']",1rff5WJrSljfCLYFoXkDiT,NaN,238200.0,0.0,Car Wash,0.620,0.179,0.0,-19.981,1.0,0.0314,0.8350,0.030000,0.0928,0.675,130.864,4.0,51.0
330458,http://www.billboard.com/charts/hot-100/1981-05-23,1981-05-23,22,I Can't Stand It,Eric Clapton And His Band,I Can't Stand ItEric Clapton And His Band,1,12.0,10,13,Eric Clapton And His Band,I Can't Stand It,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330459,http://www.billboard.com/charts/hot-100/1973-09-29,1973-09-29,22,Here I Am Come & Take Me,Al Green,Here I Am Come & Take MeAl Green,1,16.0,10,13,Al Green,Here I Am Come & Take Me,"['classic soul', 'funk', 'memphis soul', 'quiet storm', 'soul', 'soul blues', 'southern soul']",20wu6USlvWRghMOvUdfLgE,https://p.scdn.co/mp3-preview/37dbfd6ef26ecbbdaf4c9564ef8a5230767308ae?cid=b8d3901151d34489a160e3cf0ab1fa94,256693.0,0.0,Call Me,0.815,0.388,11.0,-12.238,0.0,0.0364,0.3180,0.000084,0.0673,0.846,94.558,4.0,52.0


In [6]:
perf_dance = bb_all.groupby('Performer_x').mean()['danceability'].to_frame()
perf_dance

,danceability
Performer_x,
"""Groove"" Holmes",0.553000
"""Little"" Jimmy Dickens",0.660000
"""Pookie"" Hudson",NaN
"""Weird Al"" Yankovic",0.731308
'N Sync,NaN
...,...
will.i.am & Nicki Minaj,0.854000
will.i.am Featuring Justin Bieber,0.797000
will.i.am Featuring Mick Jagger & Jennifer Lopez,0.586000


In [7]:
selected_dance = perf_dance[(perf_dance.danceability>0.577)&
                            (perf_dance.danceability<0.726)]
selected_dance

,danceability
Performer_x,
"""Little"" Jimmy Dickens",0.660000
"10,000 Maniacs",0.664130
10cc,0.583280
112,0.720282
112 Featuring Foxy Brown,0.677000
...,...
k.d. lang,0.582800
lovelytheband,0.666000
twenty one pilots,0.704686


In [8]:
perf_val = bb_all.groupby('Performer_x').mean()['valence'].to_frame()
perf_val

,valence
Performer_x,
"""Groove"" Holmes",0.524000
"""Little"" Jimmy Dickens",0.867000
"""Pookie"" Hudson",NaN
"""Weird Al"" Yankovic",0.776681
'N Sync,NaN
...,...
will.i.am & Nicki Minaj,0.686000
will.i.am Featuring Justin Bieber,0.403000
will.i.am Featuring Mick Jagger & Jennifer Lopez,0.450000


In [9]:
selected_val = perf_val[(perf_val.valence>0.394)&
                        (perf_val.valence<0.604)]
selected_val

,valence
Performer_x,
"""Groove"" Holmes",0.524000
'Til Tuesday,0.581264
0,0.580000
10cc,0.522744
1927,0.395000
...,...
twenty one pilots,0.583447
will.i.am & Britney Spears,0.501000
will.i.am Featuring Justin Bieber,0.403000


In [10]:
df_inner = pd.merge(selected_dance, selected_val, on='Performer_x', how='inner')
df_inner

,danceability,valence
Performer_x,,
10cc,0.583280,0.522744
2 Chainz Featuring Ariana Grande,0.592000,0.447000
311,0.625000,0.556000
50 Cent Featuring Eminem & Adam Levine,0.646000,0.518000
6ix9ine,0.708458,0.525250
...,...,...
benny blanco & Juice WRLD Featuring Brendon Urie,0.613000,0.532000
gnash Featuring Johnny Yukon,0.704000,0.527000
lovelytheband,0.666000,0.510000


In [11]:
perf_speechy = bb_all.groupby('Performer_x').mean()['speechiness'].to_frame()
perf_speechy

,speechiness
Performer_x,
"""Groove"" Holmes",0.032400
"""Little"" Jimmy Dickens",0.115000
"""Pookie"" Hudson",NaN
"""Weird Al"" Yankovic",0.087407
'N Sync,NaN
...,...
will.i.am & Nicki Minaj,0.059800
will.i.am Featuring Justin Bieber,0.058300
will.i.am Featuring Mick Jagger & Jennifer Lopez,0.096900


In [12]:
selected_speechy = perf_speechy[(perf_speechy.speechiness>0.053)&
                            (perf_speechy.speechiness<0.162)]
selected_speechy

,speechiness
Performer_x,
"""Little"" Jimmy Dickens",0.115000
"""Weird Al"" Yankovic",0.087407
(+44),0.123000
10 Years,0.081300
10cc,0.059845
...,...
will.i.am & Britney Spears,0.069600
will.i.am & Nicki Minaj,0.059800
will.i.am Featuring Justin Bieber,0.058300


In [13]:
df_inner2 = pd.merge(df_inner, selected_speechy, on='Performer_x', how='inner')
df_inner2

,danceability,valence,speechiness
Performer_x,,,
10cc,0.583280,0.522744,0.059845
2 Chainz Featuring Ariana Grande,0.592000,0.447000,0.089300
A Great Big World Featuring FUTURISTIC,0.610000,0.510000,0.065600
Aaron Neville,0.635313,0.571438,0.076431
Ace Hood Featuring Chris Brown,0.603000,0.561000,0.056600
...,...,...,...
Zedd & Alessia Cara,0.690000,0.544000,0.062200
benny blanco & Juice WRLD Featuring Brendon Urie,0.613000,0.532000,0.154000
gnash Featuring Johnny Yukon,0.704000,0.527000,0.090100


In [14]:
len(df_inner2)

257

In [15]:
perf_inst = bb_all.groupby('Performer_x').mean()['instrumentalness'].to_frame()
perf_inst

,instrumentalness
Performer_x,
"""Groove"" Holmes",0.876000
"""Little"" Jimmy Dickens",0.000010
"""Pookie"" Hudson",NaN
"""Weird Al"" Yankovic",0.000216
'N Sync,NaN
...,...
will.i.am & Nicki Minaj,0.000008
will.i.am Featuring Justin Bieber,0.000077
will.i.am Featuring Mick Jagger & Jennifer Lopez,0.000006


In [16]:
selected_inst = perf_inst[(perf_inst.instrumentalness>-0.006)&
                          (perf_inst.instrumentalness<0.012)]
selected_inst

,instrumentalness
Performer_x,
"""Little"" Jimmy Dickens",0.000010
"""Weird Al"" Yankovic",0.000216
(+44),0.000000
0,0.000532
10 Years,0.000208
...,...
will.i.am & Nicki Minaj,0.000008
will.i.am Featuring Justin Bieber,0.000077
will.i.am Featuring Mick Jagger & Jennifer Lopez,0.000006


In [17]:
df_inner3 = pd.merge(df_inner2, selected_inst, on='Performer_x', how='inner')
df_inner3

,danceability,valence,speechiness,instrumentalness
Performer_x,,,,
10cc,0.583280,0.522744,0.059845,0.004388
2 Chainz Featuring Ariana Grande,0.592000,0.447000,0.089300,0.000003
A Great Big World Featuring FUTURISTIC,0.610000,0.510000,0.065600,0.000000
Aaron Neville,0.635313,0.571438,0.076431,0.001501
Ace Hood Featuring Chris Brown,0.603000,0.561000,0.056600,0.000000
...,...,...,...,...
Zedd & Alessia Cara,0.690000,0.544000,0.062200,0.000000
benny blanco & Juice WRLD Featuring Brendon Urie,0.613000,0.532000,0.154000,0.000000
gnash Featuring Johnny Yukon,0.704000,0.527000,0.090100,0.000000


In [18]:
len(df_inner3)

232

In [19]:
perf_energy = bb_all.groupby('Performer_x').mean()['energy'].to_frame()
perf_energy

,energy
Performer_x,
"""Groove"" Holmes",0.487000
"""Little"" Jimmy Dickens",0.801000
"""Pookie"" Hudson",NaN
"""Weird Al"" Yankovic",0.669165
'N Sync,NaN
...,...
will.i.am & Nicki Minaj,0.667000
will.i.am Featuring Justin Bieber,0.608000
will.i.am Featuring Mick Jagger & Jennifer Lopez,0.712000


In [20]:
selected_energy = perf_energy[(perf_energy.energy>0.51)&
                              (perf_energy.energy<0.677)]
selected_energy

,energy
Performer_x,
"""Weird Al"" Yankovic",0.669165
'Til Tuesday,0.557340
0,0.572000
"10,000 Maniacs",0.665609
10cc,0.522512
...,...
twenty one pilots,0.591730
will.i.am & Nicki Minaj,0.667000
will.i.am Featuring Justin Bieber,0.608000


In [21]:
df_inner4 = pd.merge(df_inner3, selected_energy, on='Performer_x', how='inner')
df_inner4

,danceability,valence,speechiness,instrumentalness,energy
Performer_x,,,,,
10cc,0.583280,0.522744,0.059845,0.004388,0.522512
2 Chainz Featuring Ariana Grande,0.592000,0.447000,0.089300,0.000003,0.584000
Alannah Myles,0.718865,0.482000,0.054284,0.000870,0.562054
Ali Thomson,0.616500,0.551786,0.064000,0.000006,0.632786
Arizona Zervas,0.621000,0.457000,0.148000,0.000000,0.601000
...,...,...,...,...,...
Wyclef Jean,0.674000,0.452000,0.085500,0.000045,0.536000
Zedd & Alessia Cara,0.690000,0.544000,0.062200,0.000000,0.622000
benny blanco & Juice WRLD Featuring Brendon Urie,0.613000,0.532000,0.154000,0.000000,0.600000


In [22]:
perf_acous = bb_all.groupby('Performer_x').mean()['acousticness'].to_frame()
perf_acous

,acousticness
Performer_x,
"""Groove"" Holmes",0.783000
"""Little"" Jimmy Dickens",0.738000
"""Pookie"" Hudson",NaN
"""Weird Al"" Yankovic",0.132022
'N Sync,NaN
...,...
will.i.am & Nicki Minaj,0.070500
will.i.am Featuring Justin Bieber,0.001120
will.i.am Featuring Mick Jagger & Jennifer Lopez,0.104000


In [23]:
selected_acous = perf_acous[(perf_acous.acousticness>0.158)&
                            (perf_acous.acousticness<0.42)]
selected_acous

,acousticness
Performer_x,
"10,000 Maniacs",0.414965
10cc,0.340788
112 Featuring Foxy Brown,0.224000
1910 Fruitgum Co.,0.344016
2 Chainz,0.329290
...,...
a-ha,0.278537
benny blanco & Juice WRLD Featuring Brendon Urie,0.166000
dc Talk,0.166000


In [24]:
df_inner5 = pd.merge(df_inner4, selected_acous, on='Performer_x', how='inner')
df_inner5

,danceability,valence,speechiness,instrumentalness,energy,acousticness
Performer_x,,,,,,
10cc,0.583280,0.522744,0.059845,4.388146e-03,0.522512,0.340788
Alannah Myles,0.718865,0.482000,0.054284,8.702595e-04,0.562054,0.178167
Brandy,0.642388,0.565153,0.085446,2.239602e-03,0.574796,0.296791
Dru Hill,0.677326,0.491087,0.085821,0.000000e+00,0.547638,0.240246
Fabolous Featuring Ne-Yo,0.605000,0.573000,0.096700,0.000000e+00,0.614000,0.291000
Faith Evans,0.648719,0.535119,0.085249,2.429630e-07,0.586793,0.213951
Fantasia,0.655598,0.555536,0.089899,2.428557e-05,0.561278,0.360249
Ginuwine,0.690966,0.600904,0.081295,5.812895e-03,0.532379,0.204820
Gyptian,0.690000,0.497000,0.060000,2.810000e-05,0.621000,0.321000


In [25]:
len(df_inner5)

34

In [26]:
perf_live = bb_all.groupby('Performer_x').mean()['liveness'].to_frame()
perf_live

,liveness
Performer_x,
"""Groove"" Holmes",0.292000
"""Little"" Jimmy Dickens",0.627000
"""Pookie"" Hudson",NaN
"""Weird Al"" Yankovic",0.140924
'N Sync,NaN
...,...
will.i.am & Nicki Minaj,0.119000
will.i.am Featuring Justin Bieber,0.074800
will.i.am Featuring Mick Jagger & Jennifer Lopez,0.037700


In [27]:
perf_live.loc['Ne-Yo']

liveness    0.33327
Name: Ne-Yo, dtype: float64

In [28]:
selected_live = perf_live[(perf_live.liveness>0.102)&
                          (perf_live.liveness<0.219)]
selected_live

,liveness
Performer_x,
"""Weird Al"" Yankovic",0.140924
'Til Tuesday,0.169006
0,0.105000
"10,000 Maniacs",0.108765
10cc,0.171202
...,...
t.A.T.u.,0.105000
will.i.am & Britney Spears,0.131000
will.i.am & Nicki Minaj,0.119000


In [29]:
df_inner6 = pd.merge(df_inner5, selected_live, on='Performer_x', how='inner')
df_inner6

,danceability,valence,speechiness,instrumentalness,energy,acousticness,liveness
Performer_x,,,,,,,
10cc,0.583280,0.522744,0.059845,4.388146e-03,0.522512,0.340788,0.171202
Alannah Myles,0.718865,0.482000,0.054284,8.702595e-04,0.562054,0.178167,0.179784
Dru Hill,0.677326,0.491087,0.085821,0.000000e+00,0.547638,0.240246,0.164113
Fabolous Featuring Ne-Yo,0.605000,0.573000,0.096700,0.000000e+00,0.614000,0.291000,0.209000
Faith Evans,0.648719,0.535119,0.085249,2.429630e-07,0.586793,0.213951,0.172711
Fantasia,0.655598,0.555536,0.089899,2.428557e-05,0.561278,0.360249,0.104938
Halsey,0.670697,0.478192,0.057210,8.226239e-06,0.577432,0.182748,0.147645
Jaheim,0.663146,0.591683,0.104782,4.219512e-06,0.519354,0.205780,0.124622
Jazmine Sullivan,0.646028,0.589542,0.064340,0.000000e+00,0.621694,0.209658,0.130147


In [30]:
len(df_inner6)

23

In [31]:
perf_tempo = bb_all.groupby('Performer_x').mean()['tempo'].to_frame()
perf_tempo

,tempo
Performer_x,
"""Groove"" Holmes",93.66500
"""Little"" Jimmy Dickens",104.37400
"""Pookie"" Hudson",NaN
"""Weird Al"" Yankovic",127.87756
'N Sync,NaN
...,...
will.i.am & Nicki Minaj,130.09800
will.i.am Featuring Justin Bieber,128.00000
will.i.am Featuring Mick Jagger & Jennifer Lopez,106.02400


In [32]:
selected_tempo = perf_tempo[(perf_tempo.tempo>103.832)&
                            (perf_tempo.tempo<133.973)]
selected_tempo

,tempo
Performer_x,
"""Little"" Jimmy Dickens",104.374000
"""Weird Al"" Yankovic",127.877560
0,115.839000
10cc,120.636061
112 Featuring Lil'z,125.867000
...,...
will.i.am & Britney Spears,130.033000
will.i.am & Nicki Minaj,130.098000
will.i.am Featuring Justin Bieber,128.000000


In [33]:
df_inner7 = pd.merge(df_inner6, selected_tempo, on='Performer_x', how='inner')
df_inner7

,danceability,valence,speechiness,instrumentalness,energy,acousticness,liveness,tempo
Performer_x,,,,,,,,
10cc,0.583280,0.522744,0.059845,4.388146e-03,0.522512,0.340788,0.171202,120.636061
Dru Hill,0.677326,0.491087,0.085821,0.000000e+00,0.547638,0.240246,0.164113,108.566384
Faith Evans,0.648719,0.535119,0.085249,2.429630e-07,0.586793,0.213951,0.172711,120.767719
Halsey,0.670697,0.478192,0.057210,8.226239e-06,0.577432,0.182748,0.147645,129.398406
Jazmine Sullivan,0.646028,0.589542,0.064340,0.000000e+00,0.621694,0.209658,0.130147,107.368139
Jeremih,0.687239,0.555104,0.057404,0.000000e+00,0.553448,0.288418,0.207627,110.352881
Justin Bieber,0.640524,0.518942,0.119881,2.440528e-04,0.604489,0.320002,0.186610,119.468909
Lil Skies,0.683500,0.475000,0.125500,0.000000e+00,0.610000,0.382500,0.115000,129.633500
Monica,0.633815,0.561908,0.087149,2.360761e-05,0.560826,0.200023,0.211975,117.187087


In [34]:
len(df_inner7)

11

In [35]:
Halsey = bb_all[(bb_all.Performer_x == 'Justin Bieber')].drop_duplicates(subset='SongID')
Halsey

,url,WeekID,Week Position,Song_x,Performer_x,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,Performer_y,Song_y,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
379,http://www.billboard.com/charts/hot-100/2013-11-23,2013-11-23,53,Bad Day,Justin Bieber,Bad DayJustin Bieber,1,NaN,53,1,Justin Bieber,Bad Day,"['canadian pop', 'dance pop', 'pop', 'post-teen pop']",4VAngYbLewVFpcLR9FpYfR,NaN,145680.0,0.0,Journals,0.675,0.509,11.0,-6.839,0.0,0.0425,0.021500,0.000000,0.2210,0.3120,81.981,4.0,52.0
380,http://www.billboard.com/charts/hot-100/2015-12-05,2015-12-05,53,Company,Justin Bieber,CompanyJustin Bieber,1,NaN,53,1,Justin Bieber,Company,"['canadian pop', 'dance pop', 'pop', 'post-teen pop']",61uyGDPJ06MkxJtHgPmuyO,NaN,208120.0,0.0,Purpose (Deluxe),0.617,0.800,4.0,-5.024,0.0,0.0654,0.147000,0.000000,0.0852,0.4200,94.985,4.0,72.0
973,http://www.billboard.com/charts/hot-100/2012-04-14,2012-04-14,2,Boyfriend,Justin Bieber,BoyfriendJustin Bieber,1,NaN,2,1,Justin Bieber,Boyfriend,"['canadian pop', 'pop', 'post-teen pop']",3UGIZ8qcrMTwzLbx6Kttqt,NaN,171333.0,0.0,"{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'}, 'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s', 'i...",0.717,0.550,10.0,-6.019,0.0,0.0521,0.035800,0.001960,0.1260,0.3320,96.976,4.0,71.0
975,http://www.billboard.com/charts/hot-100/2015-11-14,2015-11-14,2,Sorry,Justin Bieber,SorryJustin Bieber,1,NaN,2,1,Justin Bieber,Sorry,"['canadian pop', 'pop', 'post-teen pop']",09CtPGIpYB4BrO8qb1RGsF,NaN,200786.0,0.0,"{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'}, 'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s', 'i...",0.654,0.760,0.0,-3.669,0.0,0.0450,0.079700,0.000000,0.2990,0.4100,99.945,4.0,80.0
1028,https://www.billboard.com/charts/hot-100/2021-01-16,2021-01-16,6,Anyone,Justin Bieber,AnyoneJustin Bieber,1,NaN,6,1,Justin Bieber,Anyone,"['canadian pop', 'pop', 'post-teen pop']",5MrWQZ43r6HsJeEqur1ykt,NaN,190779.0,0.0,"{'album_type': 'compilation', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'}, 'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9O...",0.686,0.538,2.0,-8.026,1.0,0.0345,0.181000,0.000003,0.1130,0.5840,115.884,4.0,0.0
1298,https://www.billboard.com/charts/hot-100/2021-04-03,2021-04-03,68,2 Much,Justin Bieber,2 MuchJustin Bieber,1,NaN,68,1,Justin Bieber,2 Much,"['canadian pop', 'pop', 'post-teen pop']",0oaY19dUwZimIgzn3ZZLZO,NaN,152796.0,0.0,"{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'}, 'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s', 'i...",0.583,0.444,9.0,-8.601,1.0,0.0456,0.593000,0.000000,0.3530,0.1670,119.590,4.0,76.0
1835,http://www.billboard.com/charts/hot-100/2013-02-16,2013-02-16,59,Nothing Like Us,Justin Bieber,Nothing Like UsJustin Bieber,1,NaN,59,1,Justin Bieber,Nothing Like Us,"['canadian pop', 'dance pop', 'pop', 'post-teen pop']",30Co9eN7JHPf1i2wEyVSMJ,NaN,199800.0,0.0,Believe Acoustic,0.425,0.156,0.0,-14.365,1.0,0.0486,0.987000,0.000051,0.1600,0.5510,166.877,4.0,67.0
1837,http://www.billboard.com/charts/hot-100/2013-12-21,2013-12-21,59,Change Me,Justin Bieber,Change MeJustin Bieber,1,NaN,59,1,Justin Bieber,Change Me,"['canadian pop', 'dance pop', 'pop', 'post-teen pop']",5cn4RhNZPPjt0onhqUMEfa,NaN,167320.0,0.0,Journals,0.304,0.340,6.0,-9.018,0.0,0.0879,0.594000,0.000000,0.1880,0.0777,72.150,5.0,49.0
2316,http://www.billboard.com/charts/hot-100/2009-10-24,2009-10-24,16,One Less Lonely Girl,Justin Bieber,One Less Lonely GirlJustin Bieber,1,NaN,16,1,Justin Bieber,One Less Lonel

In [36]:
Halsey[(Halsey.danceability>0.575)&
       (Halsey.danceability<0.725)&
       (Halsey.energy>0.51)&
       (Halsey.energy<0.67)&
       (Halsey.speechiness>0.05)&
       (Halsey.speechiness<0.15)&
       (Halsey.acousticness>0.16)&
       (Halsey.acousticness<0.42)].drop_duplicates(subset='SongID')

,url,WeekID,Week Position,Song_x,Performer_x,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,Performer_y,Song_y,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
205855,https://www.billboard.com/charts/hot-100/2020-01-18,2020-01-18,2,Yummy,Justin Bieber,YummyJustin Bieber,1,NaN,2,1,Justin Bieber,Yummy,"['canadian pop', 'pop', 'post-teen pop']",41L3O37CECZt3N7ziG2z7l,NaN,210426.0,0.0,"{'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'}, 'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s', '...",0.687,0.514,9.0,-6.612,0.0,0.0897,0.366,0.0,0.116,0.494,145.921,4.0,74.0


In [37]:
bb_all[(bb_all.danceability>0.63)].drop_duplicates(subset='SongID')

,url,WeekID,Week Position,Song_x,Performer_x,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,Performer_y,Song_y,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
14,http://www.billboard.com/charts/hot-100/1971-04-24,1971-04-24,96,Don't Knock My Love - Pt. 1,Wilson Pickett,Don't Knock My Love - Pt. 1Wilson Pickett,1,NaN,96,1,Wilson Pickett,Don't Knock My Love - Pt. 1,"['brill building pop', 'classic rock', 'classic soul', 'folk rock', 'funk', 'memphis soul', 'motown', 'rock-and-roll', 'soul', 'southern soul']",7cyLwgSVf3AnKXetNRWiTa,https://p.scdn.co/mp3-preview/5d3332b4ae616cd2157f948f7329b20470714a95?cid=b8d3901151d34489a160e3cf0ab1fa94,136400.0,0.0,The Very Best Of Wilson Pickett,0.731,0.701,7.0,-8.722,0.0,0.0287,0.1570,0.000007,0.0595,0.961,107.521,4.0,26.0
26,http://www.billboard.com/charts/hot-100/1974-07-13,1974-07-13,90,Don't Knock My Love,Diana Ross & Marvin Gaye,Don't Knock My LoveDiana Ross & Marvin Gaye,1,NaN,90,1,Diana Ross & Marvin Gaye,Don't Knock My Love,"['adult standards', 'disco', 'funk', 'mellow gold', 'motown', 'quiet storm', 'soft rock', 'soul']",4ESNSk1yvpJO3UmANTcVeI,NaN,143440.0,0.0,Diana & Marvin,0.658,0.692,9.0,-8.380,0.0,0.0467,0.1750,0.000016,0.0630,0.910,115.311,4.0,27.0
87,http://www.billboard.com/charts/hot-100/2002-09-28,2002-09-28,78,Don't Know Why,Norah Jones,Don't Know WhyNorah Jones,1,NaN,78,1,Norah Jones,Don't Know Why,"['adult standards', 'contemporary vocal jazz', 'neo mellow', 'vocal jazz']",6ybViy2qrO9sIi41EgRJgx,NaN,186146.0,0.0,Come Away With Me,0.732,0.198,5.0,-11.775,1.0,0.0280,0.8830,0.000011,0.0659,0.618,88.174,4.0,74.0
132,http://www.billboard.com/charts/hot-100/1986-12-27,1986-12-27,87,Don't Leave Me This Way,The Communards,Don't Leave Me This WayThe Communards,1,NaN,87,1,The Communards,Don't Leave Me This Way,"['dance rock', 'hi-nrg', 'new romantic', 'new wave', 'new wave pop', 'synthpop']",0ire4j2mcGovF5JA1D2cTp,https://p.scdn.co/mp3-preview/8cf6be2590b40b888c4141eb6b96ccc3e9ca8d66?cid=b8d3901151d34489a160e3cf0ab1fa94,271066.0,0.0,Communards,0.640,0.926,3.0,-5.597,1.0,0.0322,0.1350,0.000002,0.1340,0.901,131.997,4.0,65.0
147,http://www.billboard.com/charts/hot-100/1995-10-21,1995-10-21,67,Automatic Lover (Call For Love),Real McCoy,Automatic Lover (Call For Love)Real McCoy,1,NaN,67,1,Real McCoy,Automatic Lover (Call For Love),"['bubblegum dance', 'diva house', 'eurodance', 'europop', 'german techno', 'hip house']",7CEPvXcCMeCSBrMI72ayLm,https://p.scdn.co/mp3-preview/0f967dae3089c92dbd25838769cd4291662c50ea?cid=b8d3901151d34489a160e3cf0ab1fa94,227933.0,0.0,Platinum & Gold Collection,0.688,0.980,5.0,-4.862,0.0,0.0340,0.0693,0.462000,0.1600,0.538,132.992,4.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298187,http://www.billboard.com/charts/hot-100/2002-04-27,2002-04-27,83,You Know That I Love You,Donell Jones,You Know That I Love YouDonell Jones,1,NaN,83,1,Donell Jones,You Know That I Love You,"['dance pop', 'hip hop', 'hip pop', 'neo soul', 'new jack swing', 'quiet storm', 'r&b', 'urban contemporary']",5QiTs0egWAq0k2O8HWKoEP,https://p.scdn.co/mp3-preview/81afaf5bce91608aebcce7f5fef0fefe2bf2b102?cid=b8d3901151d34489a160e3cf0ab1fa94,260066.0,0.0,Life Goes On,0.767,0.494,6.0,-5.349,1.0,0.0421,0.1360,0.000277,0.0603,0.801,96.003,4.0,42.0
298231,http://www.billboard.com/charts/hot-100/2007-07-21,2007-07-21,68,You Know What It Is,T.I. Featuring Wyclef Jean,You Know What It IsT.I. Featuring Wyclef Jean,1,NaN,68,1,T.I. Featuring Wyclef Jean,You Know What It Is,"['atl hip hop', 'dance pop', 'dirty south rap', 'gangster rap', 'hip hop', 'pop rap', 'rap', 'southern hip hop', 'trap']",5Tc1KlDWyRnLLXL1y6ufsM,https://p.scdn.co/mp3-preview/4c60c597932679c45c5e97f820caf91790